#### Basic LLMChain

In [1]:
from langchain import PromptTemplate, LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [2]:
# Define the prompt template
prompt = PromptTemplate.from_template("What is the capital of {country}?")

In [3]:
# Initialize the LLM and chain
llm   = ChatOpenAI()

chain = LLMChain(llm   = llm, 
                 prompt= prompt)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_28296\2838644879.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm   = llm,


In [4]:
# Run the chain
result = chain.invoke({"country": "France"})
print(result)  # Output should be "Paris"

{'country': 'France', 'text': 'The capital of France is Paris.'}


#### SimpleSequentialChain

In [5]:
from langchain.chains import SimpleSequentialChain

In [6]:
llm_model = 'gpt-3.5-turbo'

In [7]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [8]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [9]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [10]:
product = "fitbit"

In [11]:
overall_simple_chain.run(product)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_28296\228193363.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_simple_chain.run(product)




> Entering new SimpleSequentialChain chain...
"FitTech"
FitTech is a leading provider of innovative fitness technology solutions, offering cutting-edge products and services for health and wellness enthusiasts.

> Finished chain.


'FitTech is a leading provider of innovative fitness technology solutions, offering cutting-edge products and services for health and wellness enthusiasts.'

-----------------------
#### Sequential Chain

- A SequentialChain allows you to run multiple chains in sequence, where the output of one chain is used as the input to the next.
- ---------------------

In [12]:
from langchain.chains.sequential import SequentialChain

In [13]:
# Define the prompt templates
prompt1 = PromptTemplate.from_template("What is the capital of {country}?")
prompt2 = PromptTemplate.from_template("Describe the main attractions in {city}.")

In [14]:
# Initialize the language model
llm = ChatOpenAI()

In [15]:
# Create individual chains with explicit output
chain1 = LLMChain(llm       = llm, 
                  prompt    = prompt1, 
                  output_key="city")               # Sets "city" as output for chain1

chain2 = LLMChain(llm       = llm, 
                  prompt    = prompt2, 
                  output_key= "city_description")  # Sets "city_description" as output for chain2

In [16]:
# Set up the SequentialChain
sequential_chain = SequentialChain(
    chains           = [chain1, chain2],
    input_variables  = ["country"],          # Input to the first chain
    output_variables = ["city_description"]  # Output from the last chain
)

In [17]:
# Run the SequentialChain
result = sequential_chain.invoke({"country": "Japan"})
print(result["city_description"])  # Should describe attractions in the capital city

Tokyo, the bustling capital city of Japan, is a vibrant metropolis that offers a wide range of attractions for visitors to enjoy. Some of the main attractions in Tokyo include:

1. Tokyo Tower: A symbol of the city, this iconic red and white tower offers panoramic views of the city from its observation decks.

2. Shibuya Crossing: One of the busiest pedestrian crossings in the world, this bustling intersection in Shibuya is a must-see for visitors looking to experience the energy of Tokyo.

3. Senso-ji Temple: Located in the historic Asakusa district, this ancient Buddhist temple is a popular destination for tourists and locals alike.

4. Meiji Shrine: Nestled in the heart of bustling Shibuya, this serene Shinto shrine is a peaceful oasis in the midst of the city.

5. Tsukiji Fish Market: One of the largest and most famous fish markets in the world, Tsukiji offers visitors the chance to sample some of the freshest seafood in Tokyo.

6. Tokyo Disneyland and DisneySea: These two theme pa

In [18]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm       = llm, 
                     prompt    = first_prompt, 
                     output_key= "English_Review"
                    )

In [19]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm       = llm, 
                     prompt    = second_prompt, 
                     output_key= "summary"
                    )

In [20]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm       = llm, 
                       prompt    = third_prompt,
                       output_key= "language"
                      )


In [21]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm       = llm, 
                      prompt    = fourth_prompt,
                      output_key= "followup_message"
                     )

In [22]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains          = [chain_one, chain_two, chain_three, chain_four],
    input_variables = ["Review"],
    output_variables= ["English_Review", "summary","followup_message"],
    verbose         = True
)

In [23]:
review = '''
বাংলাদেশ একটি সমৃদ্ধ ইতিহাস ও সংস্কৃতির দেশ। এই দেশে রয়েছে প্রাচীন স্থাপত্য, নদী, সবুজ প্রকৃতি এবং অসংখ্য জাতিগোষ্ঠীর বৈচিত্র্য। 
বাংলাদেশের মানুষ অতিথিপরায়ণ এবং তারা তাদের ঐতিহ্য ও সংস্কৃতিকে ভালোবাসে। প্রতি বছর এখানে বিভিন্ন ধরনের উৎসব পালন করা হয়, 
যেমন পহেলা বৈশাখ, ঈদ, দুর্গাপূজা, এবং বিজয় দিবস। এই উৎসবগুলোতে মানুষ একসঙ্গে আনন্দ করে, গান গায়, নাচে এবং খাবার খায়।

বাংলাদেশের অন্যতম বড় আকর্ষণ হচ্ছে সুন্দরবন, যা পৃথিবীর বৃহত্তম ম্যানগ্রোভ বন। এখানে রয়েল বেঙ্গল টাইগার, চিত্রা হরিণ, 
বানর এবং নানা ধরনের বন্যপ্রাণীর বসবাস। সুন্দরবনের প্রকৃতি অত্যন্ত সুন্দর এবং এটি প্রতিবছর হাজার হাজার পর্যটককে আকর্ষণ করে। 
তাছাড়া কক্সবাজার বিশ্বের দীর্ঘতম সমুদ্রসৈকত হিসাবে পরিচিত। সাদা বালুর সৈকত, নীল পানি, এবং সূর্যাস্তের দৃশ্য প্রতিটি দর্শকের মনে দাগ কাটে।

বাংলাদেশের মানুষের জীবনধারা খুবই সরল এবং প্রকৃতির সাথে গভীরভাবে সংযুক্ত। বেশিরভাগ মানুষ গ্রামাঞ্চলে বাস করে এবং তারা 
কৃষিকাজে ব্যস্ত থাকে। ধান, পাট, চা, এবং মাছ বাংলাদেশের প্রধান পণ্য। দেশটি ধীরে ধীরে শিল্প এবং প্রযুক্তিতে এগিয়ে যাচ্ছে এবং এশিয়ার 
একটি গুরুত্বপূর্ণ অর্থনৈতিক কেন্দ্রে পরিণত হচ্ছে।
'''

In [24]:
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': '\nবাংলাদেশ একটি সমৃদ্ধ ইতিহাস ও সংস্কৃতির দেশ। এই দেশে রয়েছে প্রাচীন স্থাপত্য, নদী, সবুজ প্রকৃতি এবং অসংখ্য জাতিগোষ্ঠীর বৈচিত্র্য। \nবাংলাদেশের মানুষ অতিথিপরায়ণ এবং তারা তাদের ঐতিহ্য ও সংস্কৃতিকে ভালোবাসে। প্রতি বছর এখানে বিভিন্ন ধরনের উৎসব পালন করা হয়, \nযেমন পহেলা বৈশাখ, ঈদ, দুর্গাপূজা, এবং বিজয় দিবস। এই উৎসবগুলোতে মানুষ একসঙ্গে আনন্দ করে, গান গায়, নাচে এবং খাবার খায়।\n\nবাংলাদেশের অন্যতম বড় আকর্ষণ হচ্ছে সুন্দরবন, যা পৃথিবীর বৃহত্তম ম্যানগ্রোভ বন। এখানে রয়েল বেঙ্গল টাইগার, চিত্রা হরিণ, \nবানর এবং নানা ধরনের বন্যপ্রাণীর বসবাস। সুন্দরবনের প্রকৃতি অত্যন্ত সুন্দর এবং এটি প্রতিবছর হাজার হাজার পর্যটককে আকর্ষণ করে। \nতাছাড়া কক্সবাজার বিশ্বের দীর্ঘতম সমুদ্রসৈকত হিসাবে পরিচিত। সাদা বালুর সৈকত, নীল পানি, এবং সূর্যাস্তের দৃশ্য প্রতিটি দর্শকের মনে দাগ কাটে।\n\nবাংলাদেশের মানুষের জীবনধারা খুবই সরল এবং প্রকৃতির সাথে গভীরভাবে সংযুক্ত। বেশিরভাগ মানুষ গ্রামাঞ্চলে বাস করে এবং তারা \nকৃষিকাজে ব্যস্ত থাকে। ধান, পাট, চা, এবং মাছ বাংলাদেশের প্রধান পণ্য। দেশটি ধীরে ধীরে শিল্প এবং প্রযু

#### Router chain

In [25]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [26]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [27]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [28]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [29]:
destination_chains = {}

for p_info in prompt_infos:
    name            = p_info["name"]
    prompt_template = p_info["prompt_template"]
    
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    
    chain  = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [30]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [31]:
# when LLM cant decide which chain to use
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [32]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a 
language model select the model prompt best suited for the input. 

You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 

You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string  name of the prompt to use or "DEFAULT"
    "next_inputs": string  a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [33]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

print(router_template)

Given a raw text input to a 
language model select the model prompt best suited for the input. 

You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 

You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string  name of the prompt to use or "DEFAULT"
    "next_inputs": string  a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics: Good for answering questions about physics
math: Good for answering math questions
Histor

In [35]:
router_prompt = PromptTemplate(
    template       = router_template,
    input_variables= ["input"],
    output_parser  = RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [36]:
chain = MultiPromptChain(router_chain      =router_chain, 
                         destination_chains=destination_chains, 
                         default_chain     =default_chain, 
                         verbose           =True
                        )

C:\Users\bhupe\AppData\Local\Temp\ipykernel_28296\1422575192.py:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain      =router_chain,


In [37]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [38]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [39]:
chain.run("What is the capital of Karnatake state in India?")



> Entering new MultiPromptChain chain...
None: {'input': 'What is the capital of Karnatake state in India?'}
> Finished chain.


'The capital of Karnataka state in India is Bangalore (Bengaluru).'

#### Memories with the chains

----------------------------
#### Simple Memory Chain
- Adding memory to a chain lets it remember parts of the conversation. Here, we use a ConversationChain that retains the entire dialogue context.
- -------------------------------

In [40]:
from langchain.memory import SimpleMemory

In [41]:
# Initialize the language model
llm = ChatOpenAI()

In [43]:
# Set up Simple Memory
memory = SimpleMemory()

In [44]:
user_input = "Name the Country where Bangalore city is."

In [46]:
# Access the memory to get previous interactions
previous_memory = memory.load_memory_variables({"input": user_input})  # This accesses the stored memory directly

# Construct the context from previous interactions
context = ""
for key, value in previous_memory.items():
    context += f"User: {key}\nBot: {value}\n"

context

''

In [47]:
# Define a prompt template that uses both current input and memory context
prompt_template = PromptTemplate.from_template(
    f'''You are a helpful assistant.  
    
     Here are the previous interactions: {context}
     
     User: {user_input}
     Bot:
     '''
)

prompt_template

PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.  \n    \n     Here are the previous interactions: \n     \n     User: Name the Country where Bangalore city is.\n     Bot:\n     ')

In [48]:
# Create an LLMChain
chain = LLMChain(llm   = llm, 
                 prompt= prompt_template)

In [49]:
# Construct the input for the chain using the current input and memory context
combined_input = {"context": context.strip(), "input": user_input}
combined_input

{'context': '', 'input': 'Name the Country where Bangalore city is.'}

In [50]:
# Get response from the chain
response = chain.invoke(combined_input)

In [51]:
# Display the response
print("Bot:", response)

Bot: {'context': '', 'input': 'Name the Country where Bangalore city is.', 'text': 'Bangalore city is located in the country of India.'}


In [52]:
# Save user input and bot response to memory
memory.save_context({"input": user_input}, {"output": response})

In [53]:
memory.json()

'{"memories":{}}'

In [54]:
memory.memories

{}

In [55]:
memory.to_json()

{'lc': 1,
 'type': 'not_implemented',
 'id': ['langchain', 'memory', 'simple', 'SimpleMemory'],
 'repr': 'SimpleMemory()'}

#### ConversationBufferMemory
- with simple LLMChain

In [56]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [57]:
llm = ChatOpenAI()

In [58]:
memory = ConversationBufferMemory()

C:\Users\bhupe\AppData\Local\Temp\ipykernel_28296\995385594.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [59]:
# Get user input
user_input = "Where is Bangalore?"

In [60]:
# Load previous memory (initially empty)
previous_memory = memory.load_memory_variables({}) 

In [61]:
previous_memory.get('history', "")

''

In [62]:
context = previous_memory.get('history', "")
context

''

In [63]:
# Create a prompt using context and user input
prompt_template = PromptTemplate.from_template(
    f'''You are a helpful assistant.  
    
    Here are the previous interactions: {context.strip()}
    
    User: {user_input}
    Bot:
    '''
)

In [64]:
# Create an LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)

In [65]:
# Get response from the chain
combined_input = {"context": context.strip(), "input": user_input}

response = chain.invoke(combined_input)

In [66]:
# Display the response
print("Bot:", response)

Bot: {'context': '', 'input': 'Where is Bangalore?', 'text': 'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major hub for technology and innovation.'}


In [67]:
memory.save_context({"input": user_input}, {"output": str(response)})

In [68]:
import json

# Directly pretty-print the dictionary with indentation
print(json.dumps(memory.to_json(), indent=4))

{
    "lc": 1,
    "type": "not_implemented",
    "id": [
        "langchain",
        "memory",
        "buffer",
        "ConversationBufferMemory"
    ],
    "repr": "ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Where is Bangalore?', additional_kwargs={}, response_metadata={}), AIMessage(content=\"{'context': '', 'input': 'Where is Bangalore?', 'text': 'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major hub for technology and innovation.'}\", additional_kwargs={}, response_metadata={})]))"
}


load from the memory ...

In [69]:
# Load previous memory (initially empty)
context = dict(memory.load_memory_variables({}))

In [70]:
context

{'history': "Human: Where is Bangalore?\nAI: {'context': '', 'input': 'Where is Bangalore?', 'text': 'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major hub for technology and innovation.'}"}

In [71]:
memory.buffer

"Human: Where is Bangalore?\nAI: {'context': '', 'input': 'Where is Bangalore?', 'text': 'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major hub for technology and innovation.'}"

In [72]:
memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='Where is Bangalore?', additional_kwargs={}, response_metadata={}), AIMessage(content="{'context': '', 'input': 'Where is Bangalore?', 'text': 'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major hub for technology and innovation.'}", additional_kwargs={}, response_metadata={})])

| Feature             | `LLMChain`                        | `ConversationChain`                     |
|---------------------|-----------------------------------|-----------------------------------------|
| **Purpose**         | Single interactions, stateless tasks | Multi-turn conversations, context maintenance |
| **Structure**       | Prompt + LLM                       | Prompt + LLM + Conversation history    |
| **State**           | Stateless                          | Stateful (maintains conversation history) |
| **Use Case**        | Text generation, classification, independent Q&A | Chatbots, conversational applications with context |


In [73]:
llm = ChatOpenAI()

In [74]:
prompt_template = PromptTemplate(input_variables= ["context", "input"], 
                                 template       = "{context}\nUser: {input}\nAI:")

In [75]:
# Initialize LLMChain with the LLM and template
chain = LLMChain(llm   = llm, 
                 prompt= prompt_template)

In [76]:
# Initialize an empty context
context = ""

In [77]:
# Example interactions
user_inputs = ["What is the capital of France?", 
               "And its population?", 
               "What about Germany?",
               "Compare the 2 citites in terms of job opportunities"
              ]

In [78]:
from pprint import pprint

In [79]:
for user_input in user_inputs:
    # Run the LLM chain, adding context
    response = chain.invoke({"context": context, "input": user_input})
    print(user_input)
    pprint(response)
    print("-----------------")

    # Update context with user input and LLM response
    context += f"\nUser: {user_input}\nAI: {response}"

What is the capital of France?
{'context': '',
 'input': 'What is the capital of France?',
 'text': 'The capital of France is Paris.'}
-----------------
And its population?
{'context': '\n'
            'User: What is the capital of France?\n'
            "AI: {'context': '', 'input': 'What is the capital of France?', "
            "'text': 'The capital of France is Paris.'}",
 'input': 'And its population?',
 'text': "{'context': '', 'input': 'And its population?', 'text': 'The "
         "population of Paris is approximately 2.2 million people.'}"}
-----------------
What about Germany?
{'context': '\n'
            'User: What is the capital of France?\n'
            "AI: {'context': '', 'input': 'What is the capital of France?', "
            "'text': 'The capital of France is Paris.'}\n"
            'User: And its population?\n'
            'AI: {\'context\': "\\nUser: What is the capital of France?\\nAI: '
            "{'context': '', 'input': 'What is the capital of France?', "
   

**Context Management**

- using deque

In [80]:
from collections import deque

In [81]:
# Initialize the LLM and prompt template
llm = ChatOpenAI()

In [82]:
prompt_template = PromptTemplate(input_variables= ["context", "input"], 
                                 template       = "{context}\nUser: {input}\nAI:")

In [83]:
# Initialize LLMChain with the LLM and template
chain = LLMChain(llm   = llm, 
                 prompt= prompt_template)

In [84]:
# Example interactions
user_inputs = ["What is the capital of France?", 
               "And its population?", 
               "What about Germany?",
               "Compare the 2 citites in terms of weather in general",
               "Which city is bigger in geographical size?"
              ]

In [87]:
# Initialize a deque for context management
context = deque(maxlen=5)  # Keep the last 5 interactions

In [88]:
def format_context(context):

    #return "\n".join(f"User: {user_input}\nAI: {llm_response}" for user_input, llm_response in context)
    """Format the context from the deque into a string."""
    formatted_context = ""
    for user_input, llm_response in context:
        formatted_context += f"User: {user_input}\nAI: {llm_response}\n"
    return formatted_context.strip()

In [89]:
for user_input in user_inputs:
     # Run the LLM chain with the formatted context
    formatted_context = format_context(context)
    
    response = chain.invoke({"context": context, "input": user_input})
    
    print(user_input)
    pprint(response)
    print("-----------------")

    # Update context with the new interaction
    context.append((user_input, response['text']))

What is the capital of France?
{'context': deque([], maxlen=5),
 'input': 'What is the capital of France?',
 'text': 'The capital of France is Paris.'}
-----------------
And its population?
{'context': deque([('What is the capital of France?',
                    'The capital of France is Paris.')],
                  maxlen=5),
 'input': 'And its population?',
 'text': 'The population of Paris is approximately 2.2 million people.'}
-----------------
What about Germany?
{'context': deque([('What is the capital of France?',
                    'The capital of France is Paris.'),
                   ('And its population?',
                    'The population of Paris is approximately 2.2 million '
                    'people.')],
                  maxlen=5),
 'input': 'What about Germany?',
 'text': "I'm sorry, I do not have that information."}
-----------------
Compare the 2 citites in terms of weather in general
{'context': deque([('What is the capital of France?',
                    'T

#### Using a Simple List for Context Management

In [90]:
from collections import defaultdict

In [91]:
# Initialize the LLM and prompt template
llm = ChatOpenAI()

prompt_template = PromptTemplate(
    input_variables=["context", "input"],
    template="{context}\nUser: {input}\nAI:"
)

# Initialize LLMChain with the LLM and template
chain = LLMChain(llm=llm, prompt=prompt_template)

# Example interactions
user_inputs = [
    "What is the capital of France?",
    "And its population?",
    "What about Germany?",
    "Compare the 2 cities in terms of weather in general",
    "Which city is bigger in geographical size?"
]

In [92]:
# Initialize a list for context management
context = []  # Use a simple list to keep track of interactions

In [93]:
def format_context(context):
    """Format the context from the list into a string."""
    formatted_context = ""
    for user_input, llm_response in context:
        formatted_context += f"User: {user_input}\nAI: {llm_response}\n"
    return formatted_context.strip()

In [94]:
for user_input in user_inputs:
    # Run the LLM chain with the formatted context
    formatted_context = format_context(context)

    # Invoke the chain
    response = chain.invoke({"context": formatted_context, "input": user_input})

    print(f"User: {user_input}")
    pprint(response)
    print("-----------------")

    # Update context with the new interaction
    context.append((user_input, response['text']))


User: What is the capital of France?
{'context': '',
 'input': 'What is the capital of France?',
 'text': 'The capital of France is Paris.'}
-----------------
User: And its population?
{'context': 'User: What is the capital of France?\n'
            'AI: The capital of France is Paris.',
 'input': 'And its population?',
 'text': 'The population of Paris is approximately 2.2 million people.'}
-----------------
User: What about Germany?
{'context': 'User: What is the capital of France?\n'
            'AI: The capital of France is Paris.\n'
            'User: And its population?\n'
            'AI: The population of Paris is approximately 2.2 million people.',
 'input': 'What about Germany?',
 'text': 'The capital of Germany is Berlin, and its population is around 3.7 '
         'million people.'}
-----------------
User: Compare the 2 cities in terms of weather in general
{'context': 'User: What is the capital of France?\n'
            'AI: The capital of France is Paris.\n'
            '

#### counting tokens at each interaction

In [99]:
from langchain.callbacks import get_openai_callback
from langchain.chains import ConversationChain

In [100]:
# Initialize the LLM and prompt template
llm = ChatOpenAI()

In [101]:
# Initialize memory
memory = ConversationBufferMemory()

In [102]:
# Initialize LLMChain with the LLM, template, and memory
chain = ConversationChain(llm   = llm, 
                          #prompt= prompt_template,  
                          memory= memory)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_28296\706016320.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(llm   = llm,


In [103]:
# Example interactions
user_inputs = [
    "What is the capital of China?",
    "And its population?",
    "What about India?",
    "Compare the 2 cities in terms of weather in general",
    "Which city is bigger in geographical size?",
    "How do the cultures of these places differ?",
    "What languages are predominantly spoken in these locations?",
    "What is the general cost of living like in each place?",
    "How do the healthcare systems compare between these regions?",
    "What environmental challenges do these areas face?",
    "How have these locations changed over the last few decades?"
]

In [104]:
for user_input in user_inputs:

    with get_openai_callback() as cb:
        # Invoke the chain
        response = chain.invoke({"input": user_input})
        
    
    print(f"User: {user_input}")
    pprint(response)
    print(f'>>>> Spent a total of {cb.total_tokens} tokens')
    print("---------------------------------------------------------------------")

User: What is the capital of China?
{'history': '',
 'input': 'What is the capital of China?',
 'response': 'The capital of China is Beijing. It is one of the oldest cities '
             'in the world and has a population of over 21 million people. '
             'Beijing is known for its historic landmarks such as the '
             'Forbidden City, Tiananmen Square, and the Great Wall of China. '
             'It is also a major political, cultural, and educational center '
             'in China.'}
>>>> Spent a total of 138 tokens
---------------------------------------------------------------------
User: And its population?
{'history': 'Human: What is the capital of China?\n'
            'AI: The capital of China is Beijing. It is one of the oldest '
            'cities in the world and has a population of over 21 million '
            'people. Beijing is known for its historic landmarks such as the '
            'Forbidden City, Tiananmen Square, and the Great Wall of China. It '

In [105]:
print(chain.memory.buffer)

Human: What is the capital of China?
AI: The capital of China is Beijing. It is one of the oldest cities in the world and has a population of over 21 million people. Beijing is known for its historic landmarks such as the Forbidden City, Tiananmen Square, and the Great Wall of China. It is also a major political, cultural, and educational center in China.
Human: And its population?
AI: As of 2021, the population of Beijing is estimated to be over 21 million people. The city has experienced rapid population growth in recent years due to urbanization and migration from other parts of China.
Human: What about India?
AI: India's capital is New Delhi. It is a bustling metropolis with a population of over 16 million people. New Delhi is known for its rich history, diverse culture, and iconic landmarks such as the Red Fort, India Gate, and Qutub Minar. It is also the political center of India, housing important government buildings and institutions.
Human: Compare the 2 cities in terms of wea

#### Pros and Cons of Storing Everything in Context

| Pros                                                                 | Cons                                                                     |
|----------------------------------------------------------------------|--------------------------------------------------------------------------|
| Storing everything gives the LLM the maximum amount of information. | More tokens mean slower response times and higher costs.                |
| Storing everything is simple and intuitive.                         | Long conversations cannot be remembered as we hit the LLM token limit (4096 tokens for text-davinci-003 and gpt-3.5-turbo). |
| Enhanced continuity in conversation, leading to more coherent responses. | Risk of overwhelming the model with irrelevant or outdated information, leading to potential confusion. |
| Ability to reference previous interactions verbatim, improving specificity. | Increased complexity in managing context, especially in dynamic or multi-turn conversations. |
| Greater flexibility in handling varied topics and user requests.     | Need for careful context management to avoid exceeding token limits.    |

#### Additional Considerations

- **Memory Management**: As conversations grow longer, it's essential to implement strategies to manage memory effectively. This may involve summarizing earlier parts of the conversation or discarding less relevant context.

- **Cost Efficiency**: Consideration of costs associated with token usage is vital. Implementing a context management system that intelligently selects which parts of the conversation to retain can help optimize both performance and costs.

- **Performance Trade-offs**: While having access to extensive context can enhance the quality of responses, it may also lead to slower processing times. Evaluating the balance between context richness and response latency is crucial, especially in applications requiring real-time interaction.

- **Relevance Filtering**: It can be beneficial to implement filtering mechanisms that prioritize storing relevant interactions or summarizing past information rather than retaining everything verbatim. This approach can help maintain context while staying within token limits.

- **User Control**: Providing users with options to adjust how much context is stored (e.g., through settings for short or long-term memory) can enhance user experience and engagement.


#### ConversationSummaryMemory

The **ConversationSummaryMemory** addresses a limitation of the ConversationBufferMemory. As conversations progress, the token count of our context history increases, potentially reaching a point where it exceeds the token limit of the language model (LLM).

**Key Features:**
- **Summarized Conversation Storage**: Instead of keeping a raw, unmodified history of the conversation, the memory stores a **summarized** version of previous conversation snippets.
- The summarization is performed by an **LLM** (Large Language Model), ensuring the most relevant information is retained while minimizing token usage.

**Functionality:**
- This memory type stores conversation snippets in a summarized form, allowing more efficient use of tokens by condensing long conversations.
- To perform the summarization, the memory constructor needs to have access to an LLM, which will handle the summarization process.

**Use Case:**
- **ConversationSummaryMemory** is useful for longer conversations where token limits are a concern, allowing the model to maintain context while preventing excessive token usage by summarizing the dialogue.


In [106]:
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain

#### ConversationBufferWindowMemory

The **ConversationBufferWindowMemory** is another efficient memory type for managing conversation context. It functions like short-term memory, keeping only a set number of recent conversation exchanges while dropping older ones.

**Key Features:**
- **Windowed Raw Conversation Storage**: The memory retains the most recent conversation snippets in their **raw, unmodified form**.
- The number of stored exchanges is controlled by the `k` parameter, which defines the size of the window, i.e., how many of the latest interactions to keep in memory.
  
**Functionality:**
- The memory acts as a sliding window, keeping the last `k` interactions and dropping older ones. This reduces both the aggregate token count and the number of tokens processed per call.
- By limiting the number of exchanges stored, the model can maintain relevant context while staying within token limits.

**Use Case:**
- **ConversationBufferWindowMemory** is useful when you want to keep a recent context without overwhelming the model with the entire conversation history. It allows efficient use of tokens while retaining recent exchanges for coherent responses.


In [107]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [108]:
# Initialize memory
memory = ConversationBufferWindowMemory(k=1)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_28296\1166424007.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


#### ConversationSummaryBufferMemory

- The ConversationSummaryBufferMemory is a mix of the `ConversationSummaryMemory` and the `ConversationBufferWindowMemory`.
- It summarizes the earliest interactions in a conversation while maintaining the `max_token_limit` most recent tokens in their conversation

----------------------------------
#### Stuff chain
-----------------------------------

In [109]:
from langchain.chains import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.schema import Document  # Import the Document class

In [110]:
# Create a prompt template for summarization
summary_prompt = PromptTemplate(input_variables=["text"], template="Summarize the following text: {text}")

In [111]:
llm_chain = LLMChain(llm=llm, prompt=summary_prompt)

In [112]:
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_28296\4194767714.py:1: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  stuff_chain = StuffDocumentsChain(llm_chain=llm_chain)


In [113]:
# Define the documents (text chunks) to be summarized as instances of Document
documents = [
    Document(page_content="Artificial Intelligence (AI) has a profound impact on many industries, including healthcare and finance."),
    Document(page_content="Machine learning is a subset of AI that focuses on developing systems that can learn from data."),
    Document(page_content="Deep learning, a more complex form of machine learning, has enabled advancements in image and speech recognition.")
]

In [114]:
# Run the Stuff Chain to get a summary of all text chunks combined
summary = stuff_chain.invoke(input={"input_documents": documents})
summary

{'input_documents': [Document(metadata={}, page_content='Artificial Intelligence (AI) has a profound impact on many industries, including healthcare and finance.'),
  Document(metadata={}, page_content='Machine learning is a subset of AI that focuses on developing systems that can learn from data.'),
  Document(metadata={}, page_content='Deep learning, a more complex form of machine learning, has enabled advancements in image and speech recognition.')],
 'output_text': 'AI has had a significant impact on various industries like healthcare and finance. Machine learning, a subset of AI, involves developing systems that can learn from data. Deep learning, a more advanced form of machine learning, has led to advancements in image and speech recognition.'}

----------------------------
#### Refine chain
---------------------------

In [115]:
from langchain.chains import RefineDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI, ChatOpenAI
from langchain.schema import Document

In [116]:
initial_prompt = PromptTemplate(
    input_variables= ["page_content"], 
    template       = "Generate a brief overview based on the following information: {page_content}"
)

In [117]:
# Create a prompt template for refining the output
refine_prompt = PromptTemplate(
    input_variables = ["existing_answer", "new_information"], 
    template        = "Refine the following answer by incorporating this new information: {new_information}. Answer: {existing_answer}"
)

In [118]:
# Create an initial LLMChain for the first response
initial_chain = LLMChain(llm=llm, prompt=initial_prompt)

# Create a refine LLMChain for the refinement process
refine_chain = LLMChain(llm=llm, prompt=refine_prompt)

In [119]:
# Combine the chains into a RefineDocumentsChain
refine_documents_chain = RefineDocumentsChain(
    initial_llm_chain    = initial_chain,       # Initial processing chain
    refine_llm_chain     = refine_chain,        # Refinement processing chain
    initial_response_name= "initial_response",  # Name for the initial response variable
    verbose              = True
)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_28296\987537964.py:2: LangChainDeprecationWarning: This class is deprecated. Please see the migration guide here for a recommended replacement: https://python.langchain.com/docs/versions/migrating_chains/refine_docs_chain/
  refine_documents_chain = RefineDocumentsChain(


In [120]:
# Define the documents to refine
documents = [
    Document(page_content="Artificial Intelligence (AI) is a field that simulates human intelligence."),
    Document(page_content="Machine learning is a significant branch of AI that focuses on data analysis."),
    Document(page_content="Deep learning models, a subset of machine learning, utilize neural networks.")
]

In [121]:
initial_response = initial_chain.invoke(input={"page_content": " ".join(doc.page_content for doc in documents)})

In [122]:
print("Initial Response:", initial_response)

Initial Response: {'page_content': 'Artificial Intelligence (AI) is a field that simulates human intelligence. Machine learning is a significant branch of AI that focuses on data analysis. Deep learning models, a subset of machine learning, utilize neural networks.', 'text': 'Artificial Intelligence (AI) is a field of computer science that aims to simulate human intelligence in machines. Machine learning, a significant branch of AI, involves the development of algorithms that allow computers to learn from and make predictions or decisions based on data. Deep learning models, a subset of machine learning, utilize neural networks to process and analyze large amounts of data, leading to more advanced and complex learning capabilities. This technology has been applied in various industries, including healthcare, finance, and marketing, to improve processes and decision-making.'}


In [123]:
# Accessing the output based on its structure
existing_answer = initial_response['text'] 

In [124]:
new_information = "AI technologies are rapidly advancing and are applied in various industries."

In [125]:
refine_inputs = {
    "existing_answer": existing_answer,  # Use the initial response output
    "new_information": new_information
}

In [126]:
final_refined_response = refine_chain.invoke(input=refine_inputs)

In [127]:
print("Final Refined Summary:", final_refined_response)

Final Refined Summary: {'existing_answer': 'Artificial Intelligence (AI) is a field of computer science that aims to simulate human intelligence in machines. Machine learning, a significant branch of AI, involves the development of algorithms that allow computers to learn from and make predictions or decisions based on data. Deep learning models, a subset of machine learning, utilize neural networks to process and analyze large amounts of data, leading to more advanced and complex learning capabilities. This technology has been applied in various industries, including healthcare, finance, and marketing, to improve processes and decision-making.', 'new_information': 'AI technologies are rapidly advancing and are applied in various industries.', 'text': 'Artificial Intelligence (AI) is a rapidly advancing field of computer science that aims to simulate human intelligence in machines. Machine learning, a significant branch of AI, involves the development of algorithms that allow computers

-------------------------
#### Map reduce chain
---------------------

| Feature                | MapReduce Chain                                      |
|------------------------|-----------------------------------------------------|
| **What It Is**        | A chain that processes a large dataset by mapping a function over individual items (documents) and then reducing the results to a final output. This method is particularly useful for parallel processing and handling large volumes of data. |
| **Advantages**         | - **Scalability**: Efficiently handles large datasets by dividing the workload.<br>- **Parallel Processing**: Can process multiple documents simultaneously, speeding up computations.<br>- **Flexibility**: The mapping and reducing functions can be customized to fit specific tasks, allowing for tailored processing of data. |
| **Disadvantages**      | - **Complexity**: Can be more complex to set up compared to simpler chains.<br>- **Overhead**: The overhead of managing multiple processes and aggregating results can lead to increased resource usage.<br>- **Dependency on Quality**: The quality of the final output heavily relies on the effectiveness of both the mapping and reducing steps. |
| **When to Use**       | - When working with large datasets that need to be processed in parallel.<br>- Suitable for applications like document classification, summarization, or any task that benefits from dividing the input into smaller, manageable pieces. |
